## import dependencies

In [1]:
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

import numpy as np
import random
import os

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

## type of spaces

In [3]:
Discrete(3).sample()

2

In [5]:
Box(0,1,shape=(3,3)).sample()

array([[0.8163275 , 0.28310665, 0.60204965],
       [0.16640067, 0.44635448, 0.00939645],
       [0.84363395, 0.3770904 , 0.3748703 ]], dtype=float32)

In [4]:
Tuple((Discrete(3),Box(0,1,shape=(3,3)))).sample()

(0,
 array([[0.6311409 , 0.83785903, 0.60344934],
        [0.29823706, 0.44889823, 0.55566454],
        [0.56186414, 0.2148044 , 0.50002676]], dtype=float32))

In [5]:
Dict({'height':Discrete(2),'speed':Box(0,100,shape=(1,))}).sample()

OrderedDict([('height', 1), ('speed', array([90.01421], dtype=float32))])

In [6]:
MultiBinary(4).sample()

array([0, 1, 0, 0], dtype=int8)

In [7]:
MultiDiscrete([5,2,2]).sample()

array([1, 0, 1], dtype=int64)

In [11]:
Box(low=np.array([0]),high=np.array([100])).sample()

c:\Users\Yuming\Documents\ResMan\man\lib\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


array([53.695873], dtype=float32)

## building an environment

In [11]:

class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=0, high=100,shape=(1,),dtype=float32)
        self.state = 38 + random.randint(-3,3)
        self.shower_length = 60
        pass

    def step(self, action):
        self.state += action -1
        
        self.shower_length -=1
        if self.state >=37 and self.state <= 39:
            reward = 1
        else:
            reward = -1
            
        if self.shower_length <= 0:
            done = True
        else:
            done = False
            
        info = {}
        
        return self.state, reward, done, info
        
        pass

    def render(self):
        pass

    def reset(self):
        self.state = np.array([38+random.randint(-3,3)]).astype(float)
        self.shower_length = 60
        pass


In [12]:
env = ShowerEnv()

In [4]:
env.observation_space.sample()

array([4.3058224], dtype=float32)

In [16]:
env.action_space.sample()

0

In [8]:
episodes = 5
for episode in range(episodes):
    state = env.reset()
    done = False
    score = 0

    while not done:
        # env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} ,Score:{}'.format(episode, score))
env.close()


Episode:0 ,Score:-40
Episode:1 ,Score:-30
Episode:2 ,Score:-42
Episode:3 ,Score:-20
Episode:4 ,Score:-22


## train model

In [16]:
env = ShowerEnv()
log_path = os.path.join('train', 'logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)
model.learn(total_timesteps=10000)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to train\logs\PPO_15


ValueError: Expected parameter logits (Tensor of shape (1, 3)) of distribution Categorical(logits: torch.Size([1, 3])) to satisfy the constraint IndependentConstraint(Real(), 1), but found invalid values:
tensor([[nan, nan, nan]])

In [ ]:
model.save(os.path.join('train','models','shower'))